In [18]:
!pip install apache-airflow

In [19]:
!pip install apache-airflow[postgres]==<desired_version>

/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `pip install apache-airflow[postgres]==<desired_version>'


In [20]:
!pip install --upgrade apache-airflow

In [21]:
!pip install apache-airflow-providers-postgres

In [22]:
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime
import requests
import pandas as pd

<ipython-input-22-73263d18f7e9>:2 DeprecationWarning: The `airflow.operators.python_operator.PythonOperator` class is deprecated. Please use `'airflow.operators.python.PythonOperator'`.

In [23]:
db_connection = {
    'host': '127.0.0.1',
    'port': 5432,
    'database': 'postgres',
    'user': 'postgres',
    'password': 'bunga123'
}

In [32]:
def extract_data():
    api_url = "https://berita-indo-api-next.vercel.app/api/cnn-news/teknologi"
    response = requests.get(api_url)

    if response.status_code == 200:
        data_json = response.json()
        print("API Request Successful")
    else:
        print("Error in API Request:", response.status_code)
        return None


In [44]:
# Cek apakah data ada di dalam data JSON
if 'data' in data_json:
    # Ambil value dari key data
    data_list = data_json['data']

    # Transformasi menjadi DataFrame
    df = pd.DataFrame(data_list)

    # Tampilkan DataFrame sebagai tabel dengan IPython.display
    try:
        from IPython.display import display
        display(df)
    except ImportError:
        print(df.to_markdown())
else:
    print("Error: 'data' key not found in JSON")
    df = None

,title,link,contentSnippet,isoDate,image
0,Zulhas Akui Tak Anti Asing Buntut Larangan S-C...,https://www.cnnindonesia.com/teknologi/2023110...,Zulhas menegaskan pemerintah tidak anti asing ...,2023-11-09T12:34:39.000Z,{'small': 'https://akcdn.detik.net.id/visual/2...
1,Akademisi Jawab Soal Mimpi Mahasiswa RI Berkar...,https://www.cnnindonesia.com/teknologi/2023110...,Para dosen dan praktisi teknologi menjawab soa...,2023-11-09T12:14:15.000Z,{'small': 'https://akcdn.detik.net.id/visual/2...
2,"Grab Akui Pionir Kendaraan Listrik, Pangkas 10...",https://www.cnnindonesia.com/teknologi/2023110...,Grab Indonesia mengungkap telah memangkas 10 r...,2023-11-09T11:43:08.000Z,{'small': 'https://akcdn.detik.net.id/visual/2...
3,Dirut BAKTI Kominfo Cerita Soal Keputusan Bang...,https://www.cnnindonesia.com/teknologi/2023110...,Direktur Utama Badan Aksesibilitas Telekomunik...,2023-11-09T11:21:49.000Z,{'small': 'https://akcdn.detik.net.id/visual/2...
4,BAKTI Ungkap Angka-angka Menjanjikan Imbas Tra...,https://www.cnnindonesia.com/teknologi/2023110...,Direktur Utama BAKTI Kominfo Fadhilah Mathar m...,2023-11-09T11:14:44.000Z,{'small': 'https://akcdn.detik.net.id/visual/2...
...,...,...,...,...,...
95,BMKG Ungkap Rahasia Kupang Bertahan dari Gempa...,https://www.cnnindonesia.com/teknologi/2023110...,"Gempa Magnitudo 6,3 tak membuat banyak kerusak...",2023-11-02T04:30:39.000Z,{'small': 'https://akcdn.detik.net.id/visual/2...
96,Penipuan Mumi 'Putri Duyung' Jepang Terbongkar...,https://www.cnnindonesia.com/teknologi/2023110...,"Mumi 'putri duyung' Jepang, yang menuai perdeb...",2023-11-02T03:20:09.000Z,{'small': 'https://akcdn.detik.net.id/visual/2...
97,Microsoft Buka Suara Soal Potensi Kasus Winnie...,https://www.cnnindonesia.com/teknologi/2023110...,Microsoft menjawab soal potensi insiden sensor...,2023-11-02T02:17:04.000Z,{'small': 'https://akcdn.detik.net.id/visual/2...
98,Cara Menggunakan ChatGPT di Laptop dengan Muda...,https://www.cnnindonesia.com/teknologi/2023102...,ChatGPT adalah machine learning dengan kecerda...,2023-11-02T02:00:43.000Z,{'small': 'https://akcdn.detik.net.id/visual/2...


In [33]:
def transform_data(**kwargs):
    ti = kwargs['ti']
    df = ti.xcom_pull(task_ids='extract_data_task')

In [39]:
if 'isoDate' in df.columns:
    df['isoDate'] = pd.to_datetime(df['isoDate'])
    print("Transformasi 'isoDate' berhasil")
else:
    print("Error: 'isoDate' kolom tidak ada di DataFrame")

Transformasi 'isoDate' berhasil


In [40]:
dag = DAG(
    'my_data_extraction_dag',
    start_date=datetime(2023, 1, 1),
    schedule_interval='@daily',
)

<ipython-input-40-a09148b3e859>:1 RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead.

In [41]:
extract_data_task = PythonOperator(
    task_id='extract_data_task',
    python_callable=extract_data,
    dag=dag,
)

In [42]:
transform_data_task = PythonOperator(
    task_id='transform_data_task',
    python_callable=transform_data,
    provide_context=True,
    dag=dag,
)


extract_data_task >> transform_data_task

<ipython-input-42-c5124d85c8a8>:1 RemovedInAirflow3Warning: provide_context is deprecated as of 2.0 and is no longer required

<Task(PythonOperator): transform_data_task>